In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import supervision as sv

class TripleRiderDetector:
    def __init__(self, model_path="C:/Users/DELL/Desktop/only traffic project/yolov8/yolov8x.pt", conf_threshold=0.5, iou_threshold=0.45):
        """
        Initialize the Triple Rider Detector
        Args:
            model_path: Path to YOLOv8x model weights
            conf_threshold: Confidence threshold for detections
            iou_threshold: IOU threshold for NMS
        """
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        
        # Class IDs for motorcycle and person in COCO dataset
        self.MOTORCYCLE_CLASS_ID = 3
        self.PERSON_CLASS_ID = 0
        
        # Colors for bounding boxes (BGR format)
        self.GREEN = (0, 255, 0)
        self.RED = (0, 0, 255)

    def count_riders_on_motorcycle(self, motorcycle_box, person_boxes):
        """
        Count the number of riders on a motorcycle based on IoU
        Args:
            motorcycle_box: Bounding box of motorcycle [x1, y1, x2, y2]
            person_boxes: List of person bounding boxes
        Returns:
            int: Number of riders on the motorcycle
        """
        rider_count = 0
        for person_box in person_boxes:
            # Calculate IoU between motorcycle and person
            intersection = self._calculate_intersection(motorcycle_box, person_box)
            if intersection > 0.1:  # If person overlaps with motorcycle
                rider_count += 1
        return rider_count

    def _calculate_intersection(self, box1, box2):
        """
        Calculate IoU between two bounding boxes
        """
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 < x1 or y2 < y1:
            return 0.0
            
        intersection = (x2 - x1) * (y2 - y1)
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        
        return intersection / box1_area

    def process_frame(self, frame):
        """
        Process a single frame to detect triple riders
        Args:
            frame: Input frame (numpy array)
        Returns:
            frame: Processed frame with annotations
        """
        # Run YOLOv8 inference
        results = self.model(frame, conf=self.conf_threshold, iou=self.iou_threshold)[0]
        
        # Extract detections
        boxes = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        confidences = results.boxes.conf.cpu().numpy()
        
        # Separate motorcycles and persons
        motorcycle_detections = [(box, conf) for box, cls, conf in zip(boxes, classes, confidences)
                               if cls == self.MOTORCYCLE_CLASS_ID]
        person_detections = [(box, conf) for box, cls, conf in zip(boxes, classes, confidences)
                           if cls == self.PERSON_CLASS_ID]
        
        # Process each motorcycle
        for motorcycle_box, motorcycle_conf in motorcycle_detections:
            rider_count = self.count_riders_on_motorcycle(motorcycle_box, [box for box, _ in person_detections])
            
            # Choose color based on rider count
            color = self.RED if rider_count > 2 else self.GREEN
            
            # Draw bounding box and label
            cv2.rectangle(frame, 
                        (int(motorcycle_box[0]), int(motorcycle_box[1])), 
                        (int(motorcycle_box[2]), int(motorcycle_box[3])), 
                        color, 2)
            
            # Add text label
            label = f"Motorcycle: {rider_count} riders ({motorcycle_conf:.2f})"
            cv2.putText(frame, label, 
                       (int(motorcycle_box[0]), int(motorcycle_box[1] - 10)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        return frame

    def process_video(self, input_path, output_path):
        """
        Process video file for triple rider detection
        Args:
            input_path: Path to input video
            output_path: Path to save output video
        """
        cap = cv2.VideoCapture(input_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Process frame
            processed_frame = self.process_frame(frame)
            
            # Write frame
            out.write(processed_frame)
            
            # Display frame (optional)
            cv2.imshow('Triple Rider Detection', processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Example usage
def main():
    # Initialize detector
    detector = TripleRiderDetector(
        model_path="C:/Users/DELL/Desktop/only traffic project/yolov8/yolov8x.pt",
        conf_threshold=0.6,  # Increased confidence threshold for better accuracy
        iou_threshold=0.45
    )
    
    # Process video
    input_video = "C:/Users/DELL/Desktop/only traffic project/triple riding count/triplesamplevideo.mp4"
    output_video = "C:/Users/DELL/Desktop/only traffic project/triple riding count/test.mp4"
    detector.process_video(input_video, output_video)

if __name__ == "__main__":
    main()


0: 480x640 5 persons, 1 car, 2 motorcycles, 35.9ms
Speed: 0.0ms preprocess, 35.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 1 car, 2 motorcycles, 35.2ms
Speed: 0.0ms preprocess, 35.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 motorcycles, 34.7ms
Speed: 2.9ms preprocess, 34.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 2 cars, 2 motorcycles, 33.7ms
Speed: 0.0ms preprocess, 33.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 1 car, 2 motorcycles, 26.9ms
Speed: 0.0ms preprocess, 26.9ms inference, 10.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 1 car, 2 motorcycles, 36.3ms
Speed: 0.0ms preprocess, 36.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 1 car, 2 motorcycles, 35.5ms
Speed: 0.0ms preprocess, 35.5ms inference, 3.3ms pos